# Investigate missing RT speed info

* Use `rt_vs_schedule` table for 5/4/22 to see what % of scheduled trips had corresponding RT info

In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(200_000_000_000)

import intake
import pandas as pd

from calitp.magics import query_sql
from E0_bus_oppor_vars import GCS_FILE_PATH, ANALYSIS_DATE

catalog = intake.open_catalog("*.yml")

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
'''
query = """
SELECT *
FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_vs_schedule_trips_may4_sample`

"""

rt_vs_sched = query_sql(query)

rt_vs_sched.to_parquet(
    f"{GCS_FILE_PATH}rt_vs_schedule_{ANALYSIS_DATE}.parquet")
'''

'\nquery = """\nSELECT *\nFROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_vs_schedule_trips_may4_sample`\n\n"""\n\nrt_vs_sched = query_sql(query)\n\nrt_vs_sched.to_parquet(\n    f"{GCS_FILE_PATH}rt_vs_schedule_{ANALYSIS_DATE}.parquet")\n'

In [3]:
bus_routes = catalog.bus_routes_aggregated_stats.read()

In [4]:
keep_cols = [
    "calitp_itp_id", "route_id", 
     "trips_all_day",
     "num_competitive",
     "mean_speed_mph"
]

route_table = bus_routes[keep_cols].drop_duplicates()

In [5]:
route_table[["calitp_itp_id", "route_id"]].drop_duplicates().shape

(465, 2)

In [6]:
route_table[route_table.num_competitive.isna()]

,calitp_itp_id,route_id,trips_all_day,num_competitive,mean_speed_mph


In [7]:
route_table[route_table.mean_speed_mph.isna()].shape

(355, 5)

In [8]:
missing_speeds = route_table[route_table.mean_speed_mph.isna()]

In [9]:
rt_vs_sched = pd.read_parquet(
    f"{GCS_FILE_PATH}rt_vs_schedule_{ANALYSIS_DATE}.parquet")

In [10]:
route_cols = ["calitp_itp_id", "route_id"]

df = pd.merge(
    missing_speeds,
    rt_vs_sched.drop_duplicates(subset=route_cols),
    on = route_cols,
    how = "inner",
    validate = "1:1"
)

In [11]:
df[df.pct_w_vp >= 0.5].shape

(65, 14)

In [12]:
df.shape

(355, 14)

In [13]:
df[df.pct_w_vp == 0].shape

(286, 14)